---
title: Step 04 - Modeling
subject: Churn Analysis
subtitle: Step 03 - Modeling - Churn Analysis
short_title: Modeling
authors:
  - name: Jane Doe
    affiliations:
      - University of California, Berkeley
      - STAT 159/259: Reproducible and Collaborative Statistical Data Science 
    orcid: 0000-0000-0000-0000
    email: jdoe@berkeley.edu
license: TBD
keywords: modeling, churn, spotify

abstract: What are the models? How are we modeling?
---

# Step 04: Modeling

Q: ...
- A: ...

Q: ...
- A: ...